### Cell 1: Global Configuration & Imports

In [ ]:
# Global configuration
IMAGE_FOLDER = r"C:\Users\samih\Documents\ShareX\Screenshots\For VS Vision\New FOV 33 x 33 - Only Leaves 3D"
images_folder = IMAGE_FOLDER
grid_size = 33
TIME_STEP = 1  # TIME STEP BETWEEN IMAGES in seconds (for quantification)
# image_step: process every Nth image (default = 1 processes every image)
default_image_step = 3

# Import dependencies
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.cm as cm
from IPython.display import HTML, display


### Cell 2: RAW Image Loading and Animation

In [ ]:
def load_raw_images(images_folder, image_step=default_image_step):
    """
    Load all RAW image files (PNG or JPG) from the folder sorted by modification time,
    then subsample based on image_step.
    """
    image_paths = sorted(glob.glob(os.path.join(images_folder, "*.png")), key=os.path.getmtime)
    if not image_paths:
        image_paths = sorted(glob.glob(os.path.join(images_folder, "*.jpg")), key=os.path.getmtime)
    # Subsample the image paths: select index 0, image_step, 2*image_step, ...
    image_paths = image_paths[::image_step]
    
    images = []
    for path in image_paths:
        img = cv2.imread(path)
        if img is None:
            print(f"Warning: Could not load image {path}")
        else:
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append(img_rgb)
    if not images:
        raise ValueError("No images were loaded. Please check your folder path and file types.")
    return images

def animate_raw_images(images, image_step=default_image_step):
    """
    Animate RAW images using Matplotlib.
    The frame title is labeled as "Time = {t} seconds", where t = (frame index)*(image_step).
    """
    fig, ax = plt.subplots()
    im_plot = ax.imshow(images[0])
    title = ax.set_title("Time = 0 seconds")
    ax.axis('off')
    
    def update(frame):
        im_plot.set_data(images[frame])
        time_label = frame * image_step  # first image is 0 sec
        title.set_text(f"Time = {time_label} seconds")
        return [im_plot, title]
    
    anim = animation.FuncAnimation(fig, update, frames=len(images), interval=500, blit=False, repeat=True)
    display(HTML(anim.to_jshtml()))
    plt.show()

# Example usage for RAW images:
raw_images = load_raw_images(images_folder, image_step=default_image_step)
animate_raw_images(raw_images, image_step=default_image_step)


### Cell 3: Classification Functions

In [ ]:
def classify_cell_color(avg_bgr):
    """
    Classify a cell based on its average BGR color.
    Returns:
      'green' for leaves,
      'orange' for fire,
      'white' for burnt/empty.
    """
    b, g, r = avg_bgr
    color_bgr = np.uint8([[avg_bgr]])
    color_hsv = cv2.cvtColor(color_bgr, cv2.COLOR_BGR2HSV)[0, 0]
    h, s, v = color_hsv
    if 30 <= h <= 90 and s > 50 and v > 50:
        return 'green'
    elif 0 <= h <= 40 and s > 60 and v > 60:
        return 'orange'
    else:
        return 'white'

def process_image_into_grid(img, grid_size=33):
    """
    Splits an image into a grid_size x grid_size grid.
    Outer border cells are set to 'black'; inner cells are classified.
    Returns:
      - A (grid_size x grid_size) NumPy array of labels.
      - A color-coded classification image (RGB; one pixel per cell).
    """
    height, width, _ = img.shape
    cell_h = height // grid_size
    cell_w = width // grid_size
    labels = []
    classified_img = np.zeros((grid_size, grid_size, 3), dtype=np.uint8)
    for row in range(grid_size):
        row_labels = []
        for col in range(grid_size):
            if row == 0 or row == grid_size - 1 or col == 0 or col == grid_size - 1:
                label = 'black'
            else:
                y_start = row * cell_h
                y_end   = (row + 1) * cell_h
                x_start = col * cell_w
                x_end   = (col + 1) * cell_w
                cell = img[y_start:y_end, x_start:x_end]
                avg_b = np.mean(cell[:, :, 0])
                avg_g = np.mean(cell[:, :, 1])
                avg_r = np.mean(cell[:, :, 2])
                label = classify_cell_color((avg_b, avg_g, avg_r))
            row_labels.append(label)
            if label == 'black':
                classified_img[row, col] = (0, 0, 0)
            elif label == 'green':
                classified_img[row, col] = (0, 255, 0)
            elif label == 'orange':
                classified_img[row, col] = (255, 165, 0)
            else:
                classified_img[row, col] = (255, 255, 255)
        labels.append(row_labels)
    return np.array(labels), classified_img


### Cell 4: Classified Image Animation

In [ ]:
def get_classified_images(images_folder, grid_size, upscale_factor=10, image_step=default_image_step):
    """
    Loads all classified images from the folder.
    It uses process_image_into_grid() to discretize each image,
    then upscales the classified image for better visualization.
    Only every nth image is processed, where n is image_step.
    Returns a list of upscaled classified images.
    """
    # Get image paths sorted by modification time (PNG or JPG)
    image_paths = sorted(glob.glob(os.path.join(images_folder, "*.png")), key=os.path.getmtime)
    if not image_paths:
        image_paths = sorted(glob.glob(os.path.join(images_folder, "*.jpg")), key=os.path.getmtime)
    
    # Subsample image paths
    image_paths = image_paths[::image_step]
    
    classified_images = []
    for path in image_paths:
        img = cv2.imread(path)
        if img is None:
            continue
        # Process the image into a grid of labels and a classified image
        _, cls_img = process_image_into_grid(img, grid_size=grid_size)
        # Upscale the classified image (e.g., each cell becomes upscale_factor x upscale_factor pixels)
        upscaled = cv2.resize(cls_img,
                              (grid_size * upscale_factor, grid_size * upscale_factor),
                              interpolation=cv2.INTER_NEAREST)
        classified_images.append(upscaled)
    return classified_images

def animate_classified_images(classified_images, image_step=default_image_step):
    """
    Animate a list of upscaled classified images.
    Each frame is labeled as "Time = {t} seconds", where t = (frame index)*image_step.
    """
    fig, ax = plt.subplots()
    im_plot = ax.imshow(classified_images[0])
    title = ax.set_title("Time = 0 seconds")
    ax.axis('off')
    
    def update(frame):
        im_plot.set_data(classified_images[frame])
        time_label = frame * image_step  # first image is time 0
        title.set_text(f"Time = {time_label} seconds")
        return [im_plot, title]
    
    anim = animation.FuncAnimation(fig, update, frames=len(classified_images),
                                   interval=500, blit=False, repeat=True)
    display(HTML(anim.to_jshtml()))
    plt.show()

# --- Example usage for animating classified images ---
classified_images = get_classified_images(images_folder, grid_size=grid_size,
                                            upscale_factor=10, image_step=default_image_step)
animate_classified_images(classified_images, image_step=default_image_step)


### Cell 5: Side‑by‑Side-by-Side Comparison (Original vs. Classified vs. Isochrone Images)

In [ ]:
def get_isochrone_contours(labels_grid, upscale_factor=10):
    """
    Create a binary mask from the classification grid for cells labeled as 
    'orange' or 'white', upscale it, and find contours.
    """
    grid_size_local = labels_grid.shape[0]
    mask = np.zeros((grid_size_local, grid_size_local), dtype=np.uint8)
    for r in range(grid_size_local):
        for c in range(grid_size_local):
            if labels_grid[r, c] in ('orange', 'white'):
                mask[r, c] = 255
    upscaled_size = (grid_size_local * upscale_factor, grid_size_local * upscale_factor)
    mask_upscaled = cv2.resize(mask, upscaled_size, interpolation=cv2.INTER_NEAREST)
    contours, _ = cv2.findContours(mask_upscaled, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def draw_isochrone_only(labels_grid, upscale_factor=10):
    """
    Create a blank white canvas and draw the isochrone contour (outline) 
    from the classification grid using thin red lines.
    """
    grid_size_local = labels_grid.shape[0]
    upscaled_size = (grid_size_local * upscale_factor, grid_size_local * upscale_factor)
    contours = get_isochrone_contours(labels_grid, upscale_factor=upscale_factor)
    outline_img = 255 * np.ones((upscaled_size[1], upscaled_size[0], 3), dtype=np.uint8)
    if contours:
        cv2.drawContours(outline_img, contours, -1, (255, 0, 0), 1)
    return outline_img

def show_three_panel_comparison(images_folder, grid_size=33, upscale_factor=10, image_step=default_image_step):
    """
    For each (or every nth) image in the folder, display a single figure with
    three panels:
      - Left: Original image (converted to RGB)
      - Middle: Classified image (discretised & upscaled)
      - Right: Isochrone-only image (firefront outline, upscaled)
      
    Time is labeled as "Time = t sec", where t = (image index * image_step),
    with the first image counted as time = 0 sec.
    """
    # Get image paths sorted by modification time and subsample using image_step
    image_paths = sorted(glob.glob(os.path.join(images_folder, "*.png")), key=os.path.getmtime)
    if not image_paths:
        image_paths = sorted(glob.glob(os.path.join(images_folder, "*.jpg")), key=os.path.getmtime)
    image_paths = image_paths[::image_step]
    
    for idx, path in enumerate(image_paths):
        img = cv2.imread(path)
        if img is None:
            print(f"Warning: Could not load image {path}")
            continue

        # Convert original image to RGB.
        orig_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Process image to obtain grid classification.
        labels_grid, classified_img = process_image_into_grid(img, grid_size=grid_size)
        # Upscale the classified image for better visualization.
        upscaled_classified = cv2.resize(classified_img,
                                         (classified_img.shape[1] * upscale_factor,
                                          classified_img.shape[0] * upscale_factor),
                                         interpolation=cv2.INTER_NEAREST)
        # Generate the isochrone-only image.
        isochrone_img = draw_isochrone_only(labels_grid, upscale_factor=upscale_factor)

        # Calculate the time label: first image is 0 sec.
        time_label = idx * image_step

        # Create a single figure with three subplots (side-by-side)
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        axes[0].imshow(orig_rgb)
        axes[0].set_title(f"Original (Time = {time_label} sec)")
        axes[0].axis('off')
        
        axes[1].imshow(upscaled_classified)
        axes[1].set_title(f"Classified (Time = {time_label} sec)")
        axes[1].axis('off')
        
        axes[2].imshow(cv2.cvtColor(isochrone_img, cv2.COLOR_BGR2RGB))
        axes[2].set_title(f"Isochrone (Time = {time_label} sec)")
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()

# Example usage: process and display three-panel comparison using the default_image_step
show_three_panel_comparison(images_folder, grid_size=grid_size, upscale_factor=10, image_step=default_image_step)


### Cell 6: Overlay of Isochrones (Colour Gradient Option)

In [ ]:
def overlay_all_isochrones_color_gradient(images_folder, grid_size=33, upscale_factor=10, line_thickness=1, image_step=default_image_step):
    """
    For all (or every nth) image in the folder, compute the firefront contours
    and overlay them on a single white canvas using a color gradient.
    Each processed image is labeled as "Time = t sec" where t = index * image_step.
    A colorbar is added as a key to map colors to time.
    """
    # Get and subsample image paths
    image_paths = sorted(glob.glob(os.path.join(images_folder, "*.png")), key=os.path.getmtime)
    if not image_paths:
        image_paths = sorted(glob.glob(os.path.join(images_folder, "*.jpg")), key=os.path.getmtime)
    if not image_paths:
        raise ValueError("No images found in the specified folder.")
    image_paths = image_paths[::image_step]
    
    # Get colormap from matplotlib
    cmap = cm.get_cmap('jet', len(image_paths))
    upscaled_size = (grid_size * upscale_factor, grid_size * upscale_factor)
    overlay_img = 255 * np.ones((upscaled_size[1], upscaled_size[0], 3), dtype=np.uint8)
    
    for i, path in enumerate(image_paths):
        img = cv2.imread(path)
        if img is None:
            print(f"Warning: Could not load image {path}")
            continue
        labels_grid, _ = process_image_into_grid(img, grid_size=grid_size)
        contours = get_isochrone_contours(labels_grid, upscale_factor=upscale_factor)
        # Get the distinct color for this time step from the colormap
        color_rgba = cmap(i)
        # Convert RGBA (0-1) to BGR (0-255) since OpenCV uses BGR order
        color_bgr = (int(color_rgba[2]*255), int(color_rgba[1]*255), int(color_rgba[0]*255))
        if contours:
            cv2.drawContours(overlay_img, contours, -1, color_bgr, line_thickness)

    # Plot the final overlay image with a colorbar key that maps time.
    fig, ax = plt.subplots(figsize=(8,8))
    # Convert BGR overlay to RGB for correct display in matplotlib.
    ax.imshow(cv2.cvtColor(overlay_img, cv2.COLOR_BGR2RGB))
    ax.set_title("Overlay of Isochrones (Color Gradient)")
    ax.axis('off')
    
    # Create a ScalarMappable for the colorbar.
    # We assume the time values run from 0 to (n-1)*image_step seconds.
    norm = plt.Normalize(vmin=0, vmax=(len(image_paths)-1)*image_step)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax, fraction=0.046, pad=0.04, ticks=np.linspace(0, (len(image_paths)-1)*image_step, num=len(image_paths)))
    cbar.set_label("Time (seconds)")
    
    plt.tight_layout()
    plt.show()

# Run overlay of isochrones with color gradient and key
overlay_all_isochrones_color_gradient(images_folder, grid_size=grid_size, upscale_factor=10, line_thickness=1, image_step=default_image_step)


### Cell 7: Fire Spread Quantification and Plotting

In [ ]:
def graph_process_image_into_grid(img, grid_size=grid_size):
    """
    Splits the input image into a grid_size x grid_size grid.
    Outer border cells are set to 'black'. Inner cells are classified.
    Returns a (grid_size x grid_size) NumPy array of labels.
    """
    height, width, _ = img.shape
    cell_h = height // grid_size
    cell_w = width // grid_size
    labels = []
    for row in range(grid_size):
        row_labels = []
        for col in range(grid_size):
            if row == 0 or row == grid_size - 1 or col == 0 or col == grid_size - 1:
                row_labels.append('black') 
            else:
                y_start = row * cell_h
                y_end   = (row + 1) * cell_h
                x_start = col * cell_w
                x_end   = (col + 1) * cell_w
                cell = img[y_start:y_end, x_start:x_end]
                avg_b = np.mean(cell[:, :, 0])
                avg_g = np.mean(cell[:, :, 1])
                avg_r = np.mean(cell[:, :, 2])
                row_labels.append(classify_cell_color((avg_b, avg_g, avg_r)))
        labels.append(row_labels)
    return np.array(labels)

# Main code to iterate through images, count areas, and plot results.
# Assumes a 1-second time step between images.
# Get all image file paths (using images_folder defined earlier)
image_paths = sorted(glob.glob(os.path.join(images_folder, '*.png')), key=os.path.getmtime)
if not image_paths:
    image_paths = sorted(glob.glob(os.path.join(images_folder, '*.jpg')), key=os.path.getmtime)
if not image_paths:
    raise ValueError("No images found in the specified folder.")

# Lists to store the area (number of cells) per image
fire_reached_area = []  # counts cells that are either 'orange' or 'white'
fire_area = []          # counts cells that are 'orange'

for path in image_paths:
    img = cv2.imread(path)
    if img is None:
        print(f"Warning: Could not load image {path}")
        continue
    grid = graph_process_image_into_grid(img, grid_size=grid_size)
    # Count cells that are either 'orange' or 'white'
    reached_count = np.sum((grid == 'orange') | (grid == 'white'))
    # Count cells that are just 'orange'
    fire_count = np.sum(grid == 'orange')
    fire_reached_area.append(reached_count)
    fire_area.append(fire_count)

# Generate time points (in seconds)
time_points = np.arange(len(fire_reached_area))  # since TIME_STEP is 1, time equals image index

# Print the values that are being plotted:
print("Time (seconds):", time_points)
print("Fire Reached Area (cells, orange+white):", fire_reached_area)
print("Fire Area (cells, orange only):", fire_area)

# Plot the total area over time
plt.figure(figsize=(10,5))
plt.plot(time_points, fire_reached_area, marker='o', label='Fire Reached Area (orange+white)')
plt.plot(time_points, fire_area, marker='s', label='Fire Area (orange)')
plt.xlabel("Time (seconds)")
plt.ylabel("Number of Cells")
plt.title("Fire Spread Area Over Time")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Compute the differences between consecutive images (new cells per second)
fire_reached_diff = np.diff(fire_reached_area)
fire_area_diff = np.diff(fire_area)

# Print the differences (rate of spread)
print("New Fire Reached Area per second (cells):", fire_reached_diff)
print("New Fire Area per second (cells):", fire_area_diff)

# Plot the rate of spread (area increase per second)
plt.figure(figsize=(10,5))
plt.plot(time_points[1:], fire_reached_diff, marker='o', label='New Fire Reached Area per Second')
plt.plot(time_points[1:], fire_area_diff, marker='s', label='New Fire Area per Second')
plt.xlabel("Time (seconds)")
plt.ylabel("New Cells per Second")
plt.title("Fire Spread Rate (Area Increase per Second)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
